In [1]:
from azureml.core import Workspace, ScriptRunConfig
from azureml.core.compute import DatabricksCompute, ComputeTarget

c:\Users\Vikas.Yetintala\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
ws= Workspace.from_config("./config")

In [ ]:
db_resource_group = ""
db_workspace_name = ""
db_access_token = ""
db_compute_name = ""

In [ ]:
if db_compute_name not in ws.compute_targets:
    attach_config = DatabricksCompute.attach_configuration(resource_group=db_resource_group,
                                                            workspace_name=db_workspace_name,
                                                            access_token=db_access_token)
    print("Attaching the compute target")
    db_cluster = ComputeTarget.attach(ws,db_compute_name,attach_config)
    db_cluster.wait_for_completion(True)
else:
    db_cluster= ws.compute_targets[db_compute_name]

In [ ]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies   

In [ ]:
myenv= Environment(name="MyEnvironment")
myenv_dep= CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults','azureml-interpret','azureml-explain-model','interpret-community'])
myenv.python.conda_dependencies = myenv_dep
myenv.register(ws)

In [ ]:
from azureml.core.runconfig import RunConfiguration 
run_config = RunConfiguration()
run_config.target= db_cluster
run_config.environment = myenv 

In [ ]:
from azureml.core import Experiment
experiment=Experiment(workspace=ws,name="azureml_databricks_step")

In [ ]:
script_config = ScriptRunConfig(source_directory=".",
                                script="TrainingScript.py",
                                environment=myenv,
                                run_config=run_config) 
new_run1 = experiment.submit(config=script_config)
new_run1.wait_for_completion()